In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from delta.tables import DeltaTable

### Data Reading

In [0]:
df = (
    spark
    .sql(
        """
        SELECT *
        FROM databricks_cata.silver.orders
        """
    )
)

In [0]:
df.display()

In [0]:
df_dimcust = (
    spark
    .sql(
        """
        SELECT DimCustomerKey, customer_id AS dim_customer_id
        FROM databricks_cata.gold.DimCustomers
        """
    )
)

In [0]:
df_dimprod = (
    spark
    .sql(
        """
        SELECT product_id AS DimProductKey, product_id As dim_product_id
        FROM databricks_cata.gold.DimCustomers
        """
    )
)

### Creating df_fact

In [0]:
df_fact = (
    df
    .join(
        other=df_dimcust,
        on=df.customer_id == df_dimcust.customer_id,
        how='left'
    )
    .join(
        other=df_dimprod,
        on=df.product_id == df_dimprod.product_id,
        how='left'
    )
)

df_fact = (
    df_fact
    .drop('dim_customer_id', 'dim_product_id')
)

In [0]:
df_fact.display()

### Upsert on Fact Table

In [0]:
if spark.catalog.tableExists('databricks_cata.gold.FactOrders'):
    target_delta_table = DeltaTable.forPath(spark, 'abfss://gold@databricksstorageete.dfs.core.windows.net/FactOrders')
    (target_delta_table
        .alias('target')
        .merge(
            source=df_fact.alias('source'),
            condition=F.expr('target.order_id  = source.order_id AND target.DimCustomerKey = src.DimCustomerKey AND target.DimProductKey = src.DimProductKey')
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )

    
else:
    (
        df_fact
        .write
        .format('delta')
        .option('path', 'dbfs:gold@databricksstorageete.dfs.core.windows.net/FactOrders')
        .saveAsTable('databricks_cata.gold.FactOrders')
    )